In [ ]:
from transformers import AutoTokenizer
from transformers import T5ForConditionalGeneration

import torch

model = T5ForConditionalGeneration.from_pretrained("castorini/monot5-3b-msmarco-10k", torch_dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained("t5-3b")

device = torch.device("cuda")
model.to(device)

In [ ]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5

reranker = MonoT5(model=model)

In [ ]:
import json

test_set = json.load(open("./data/iirc_to_rerank.json",'r'))

In [ ]:
from tqdm import tqdm

results = []

for item in tqdm(test_set):
  chosen = []
  for d in item['decomposition']:
    query = Query(d['question'])
    texts = [Text(p['contents'], {'docid': p['id'],"title":p['title'],"text":p['contents']}, 0) for p in d['documents']]
    reranked = reranker.rerank(query, texts)
    reranked.sort(key=lambda x: x.score, reverse=True)
    d['documents'] = [text.metadata for text in reranked]
  results.append(item)

In [ ]:
json.dump(results, open("iirc_reranked.json",'w'))